In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pip install roboflow

In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key="idKFK8PupLHRjgqWDlCC")
project = rf.workspace("rsdetector").project("roadsign-clf")
dataset = project.version(1).download("folder")


In [45]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Set your dataset path and parameters
train_data_dir = './roadsign-clf-2/train/'
test_data_dir = './roadsign-clf-2/valid/'
batch_size = 32
num_epochs = 10
image_size = (64, 64)
num_classes = 4  # Change this to the number of classes in your dataset


In [ ]:
import cv2

# Data augmentation for the training dataset
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    fill_mode='nearest'
)

# Load and preprocess training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
)

In [ ]:
# Preprocess test data
test_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
)

In [ ]:
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras import regularizers
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

# Create Precision and Recall objects outside the function
precision_obj = Precision()
recall_obj = Recall()

def f1_score(y_true, y_pred):
    precision_val = precision_obj(y_true, y_pred)
    recall_val = recall_obj(y_true, y_pred)
    return 2*((precision_val*recall_val)/(precision_val+recall_val+1e-5))


model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(64, 64, 3), kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Consider adding dropout after pooling layers

model.add(layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(layers.Conv2D(256, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(layers.Conv2D(256, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(layers.Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=[Recall(name="recall"), Precision(name="precision"), f1_score])

model.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=100,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:
model_save_path = './cnn/14800'
model.save(model_save_path)


In [ ]:
from tensorflow.keras.models import load_model

def f1_score(y_true, y_pred):
    precision_val = precision_obj(y_true, y_pred)
    recall_val = recall_obj(y_true, y_pred)
    return 2*((precision_val*recall_val)/(precision_val+recall_val+1e-5))

model_load_path = '/content/drive/MyDrive/ICBT BSc/RSDetector/model/cnn/13800'
final_model_recognition = load_model(model_load_path,custom_objects={'f1_score': f1_score})

In [ ]:
import matplotlib.pyplot as plt

# Create a figure with a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(12, 10))

# Plot training & validation loss values
axs[0, 0].plot(history.history['loss'], label='Training Loss')
axs[0, 0].plot(history.history['val_loss'], label='Validation Loss')
axs[0, 0].set_title('Model Loss')
axs[0, 0].set_xlabel('Epoch')
axs[0, 0].set_ylabel('Loss')
axs[0, 0].legend()

num = 8
lab_recall = 'recall'
lab_precision = 'precision'
lab_f1_score = 'f1_score'

# Plot training & validation recall values
axs[0, 1].plot(history.history['{}'.format(lab_recall)], label='Training {}'.format(lab_recall))
axs[0, 1].plot(history.history['val_{}'.format(lab_recall)], label='Validation {}'.format(lab_recall))
axs[0, 1].set_title('Model {}'.format(lab_recall))
axs[0, 1].set_xlabel('Epoch')
axs[0, 1].set_ylabel('{}'.format(lab_recall))
axs[0, 1].legend()

# Plot training & validation precision values
axs[1, 0].plot(history.history['{}'.format(lab_precision)], label='Training {}'.format(lab_precision))
axs[1, 0].plot(history.history['val_{}'.format(lab_precision)], label='Validation {}'.format(lab_precision))
axs[1, 0].set_title('Model {}'.format(lab_precision))
axs[1, 0].set_xlabel('Epoch')
axs[1, 0].set_ylabel('{}'.format(lab_precision))
axs[1, 0].legend()

# Plot training & validation precision values
axs[1, 1].plot(history.history['{}'.format(lab_f1_score)], label='Training {}'.format(lab_f1_score))
axs[1, 1].plot(history.history['val_{}'.format(lab_f1_score)], label='Validation {}'.format(lab_f1_score))
axs[1, 1].set_title('Model {}'.format(lab_f1_score))
axs[1, 1].set_xlabel('Epoch')
axs[1, 1].set_ylabel('{}'.format(lab_f1_score))
axs[1, 1].legend()


# Adjust spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()


In [61]:
lab = ['0','1','10','2','3','4','5','6','7','8','9']

In [ ]:
# Predict using the trained model
# Replace 'image_path' with the path to the image you want to predict

from tensorflow.keras.preprocessing import image
import numpy as np

image_path = './roadsign-clf-2/valid/3/N_2_117423_jpg.rf.da093e2b05bfc5a622dd4a4ec57cca33.jpg'
img = image.load_img(image_path, target_size=(64,64))
# img = img.convert('L')
img_array = image.img_to_array(img)
img_array /= 255.0  # Normalize the image
img_array = np.expand_dims(img_array, axis=0)

predictions = model.predict(img_array)
class_index = np.argmax(predictions)
# You can map the class index to class labels based on your dataset

print(f"Predicted class: {lab[class_index]}")

In [ ]:
predictions